In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from pathlib import Path
import shutil
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import cv2
import matplotlib.pylab as plt
from tqdm import tqdm

In [ ]:
# directory where train and train.csv files located (from competition data)
data_dir = Path('/content/drive/MyDrive/SartoriousDatasets/Initial')

train_dir = data_dir/'train'
train_csv = data_dir/'train.csv'

train_df = pd.read_csv(train_csv)
train_df["img_path"] = train_df["id"].apply(lambda x: os.path.join(train_dir, x + ".png"))
tmp_df = train_df.drop_duplicates(subset=['id', 'img_path']).reset_index(drop=True)
tmp_df['annotation'] = train_df.groupby('id')['annotation'].agg(list).reset_index(drop=True)

train_df = tmp_df.copy()
train_df.head()

,id,annotation,width,height,cell_type,plate_time,sample_date,sample_id,elapsed_timedelta,img_path
0,0030fd0e6378,[118145 6 118849 7 119553 8 120257 8 120961 9 ...,704,520,shsy5y,11h30m00s,2019-06-16,shsy5y[diff]_E10-4_Vessel-714_Ph_3,0 days 11:30:00,/content/drive/MyDrive/SartoriousDatasets/Init...
1,0140b3c8f445,[32499 3 33201 7 33902 9 34604 10 35306 11 360...,704,520,astro,09h00m00s,2020-09-13,astros[cereb]_F8-3_Vessel-361_Ph_4,0 days 09:00:00,/content/drive/MyDrive/SartoriousDatasets/Init...
2,01ae5a43a2ab,[241026 3 241726 9 242427 13 243130 14 243834 ...,704,520,cort,13h30m00s,2020-11-04,cort[oka-high]_B5-1_Vessel-377_Ph_1,0 days 13:30:00,/content/drive/MyDrive/SartoriousDatasets/Init...
3,026b3c2c4b32,[170753 5 171454 12 172158 13 172862 13 173565...,704,520,cort,19h30m00s,2020-11-04,cort[oka-low]_H6-2_Vessel-377_Ph_2,0 days 19:30:00,/content/drive/MyDrive/SartoriousDatasets/Init...
4,029e5b3b89c7,[139142 7 139845 10 140548 13 141251 15 141955...,704,520,cort,13h30m00s,2020-10-27,cort[pre-treat]_B8-2_Vessel-377_Ph_2,0 days 13:30:00,/content/drive/MyDrive/SartoriousDatasets/Init...


In [ ]:
# !!! Run this cell with split=True only ones to determinate KFold split (otherwise set split = False)

split = False

# Split initial set on 5 folds stratified by cell type. Each fold will containt ~ 120 samples.
# Keep 1 fold for validation purposes

if split:
    skf = StratifiedKFold(n_splits=5, shuffle=True)
    
    for fold, (_, val_idx) in enumerate(skf.split(X=train_df, y=train_df['cell_type']), 1):
        train_df.loc[val_idx, 'fold'] = fold
    train_df['fold'] = train_df['fold'].astype(np.uint8)
    
    # save id-to-fold dataset
    train_df[['id', 'fold']].to_csv('/content/drive/MyDrive/SartoriousDatasets/id_to_fold.csv', index=False)
else:
    id_to_fold = pd.read_csv('/content/drive/MyDrive/SartoriousDatasets/id_to_fold.csv')

    train_df = train_df.merge(right=id_to_fold, how='left', on='id')

print(train_df.shape)  # must be (606, 11)
train_df.head()

(606, 11)


,id,annotation,width,height,cell_type,plate_time,sample_date,sample_id,elapsed_timedelta,img_path,fold
0,0030fd0e6378,[118145 6 118849 7 119553 8 120257 8 120961 9 ...,704,520,shsy5y,11h30m00s,2019-06-16,shsy5y[diff]_E10-4_Vessel-714_Ph_3,0 days 11:30:00,/content/drive/MyDrive/SartoriousDatasets/Init...,5
1,0140b3c8f445,[32499 3 33201 7 33902 9 34604 10 35306 11 360...,704,520,astro,09h00m00s,2020-09-13,astros[cereb]_F8-3_Vessel-361_Ph_4,0 days 09:00:00,/content/drive/MyDrive/SartoriousDatasets/Init...,4
2,01ae5a43a2ab,[241026 3 241726 9 242427 13 243130 14 243834 ...,704,520,cort,13h30m00s,2020-11-04,cort[oka-high]_B5-1_Vessel-377_Ph_1,0 days 13:30:00,/content/drive/MyDrive/SartoriousDatasets/Init...,5
3,026b3c2c4b32,[170753 5 171454 12 172158 13 172862 13 173565...,704,520,cort,19h30m00s,2020-11-04,cort[oka-low]_H6-2_Vessel-377_Ph_2,0 days 19:30:00,/content/drive/MyDrive/SartoriousDatasets/Init...,3
4,029e5b3b89c7,[139142 7 139845 10 140548 13 141251 15 141955...,704,520,cort,13h30m00s,2020-10-27,cort[pre-treat]_B8-2_Vessel-377_Ph_2,0 days 13:30:00,/content/drive/MyDrive/SartoriousDatasets/Init...,5


In [ ]:
def rle_decode(mask_rle, shape):
    
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)


def get_mask(annotation, width, height, binary_mask=False):
    dtype = np.uint8 if binary_mask else np.uint16
    
    img_mask = np.zeros((height, width), dtype=dtype)
    for i, annot in enumerate(annotation):
        fill_value = 1 if binary_mask else i + 1
        img_mask = np.where(rle_decode(annot, (height, width)) != 0, fill_value, img_mask)
    
    return img_mask

  
def get_mask_w_intersection(mask, contour, annotation, width, height):
    # TODO: Overlapping cells will be involved too
    img_mask = np.zeros((height, width), dtype=np.uint8)

    for annot in annotation:
        cell_mask = rle_decode(annot, (height, width))
        img_mask += cell_mask

    img_mask = np.clip(img_mask, 0, 2)  # convert int. of > 2 cells in 1 cls

    intersection = np.where(img_mask == 2, 1, 0)
    intersection = np.where((intersection == 1) & (contour == 1), 1, 0).astype(np.uint8)

    final_mask = mask + intersection
    
    return final_mask


def extract_contour(mask):
    labels = np.unique(mask)[1:]  # exclude background
    
    contours = np.full_like(mask, fill_value=0., dtype=np.uint8)
    
    for label in labels:
        mask_l = np.where(mask == label, 1, 0).astype(np.uint8)
        contours_l, _ = cv2.findContours(mask_l, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(contours, contours_l, -1, (1, 0, 0))
        
    return contours

In [ ]:
def create_masks(dataset, mask_dir,
                 mask_type='binary', subtract_contour=False):
    
    stem_dict = {
        'binary': 'mask',
        'mask_w_contour': 'mask_w_contour',
        'categorical_intersection': 'categorical_intersection',
        'categorical': 'categorical_mask'
    }
    
    if not os.path.exists(mask_dir):
        os.mkdir(mask_dir)
        
    mask_stem = stem_dict.get(mask_type, 'mask')  # binary masks by default
    
    for _, (path, annot, width, height) in tqdm(dataset[['img_path', 'annotation', 'width', 'height']].iterrows()):
        stem = Path(path).stem
        
        mask = get_mask(annot, width, height, binary_mask=True)

        if mask_type != 'binary' and mask_type != 'mask_w_intersection':
            instance_mask = get_mask(annot, width, height, binary_mask=False)
            contour = extract_contour(instance_mask)
        
        if mask_type == 'mask_w_contour':
            if subtract_contour:
                mask = mask - contour
            final_mask = np.stack([mask, contour], axis=-1)  # [H, W, 2]
        elif mask_type == 'categorical_intersection':
            final_mask = get_mask_w_intersection(mask, contour, annot, width, height)
        elif mask_type == 'categorical':
            final_mask = mask + contour
        else:
            final_mask = mask
        
        mask_path = Path(mask_dir)/(stem + '_' + mask_stem + '.tif')
        
        Image.fromarray(final_mask).save(mask_path)

        
def create_images(dataset, image_dir):
    
    if not os.path.exists(image_dir):
        os.mkdir(image_dir)
        
    for _, (path, annot, width, height) in tqdm(dataset[['img_path', 'annotation', 'width', 'height']].iterrows()):
        stem = Path(path).stem    
        
        image_path = Path(image_dir)/(stem + '.png')
        
        shutil.copy(path, image_path)


def create_instances(dataset, instance_dir):
    
    if not os.path.exists(instance_dir):
        os.mkdir(instance_dir)
        
    for _, (path, annot, width, height) in tqdm(dataset[['img_path', 'annotation', 'width', 'height']].iterrows()):
        stem = Path(path).stem
        
        instance_mask = get_mask(annot, width, height, binary_mask=False)
        
        instance_path = Path(instance_dir)/(stem + '_' + 'instance' + '.tif')
        
        Image.fromarray(instance_mask).save(instance_path)

In [ ]:
# dir where to store all extracted data
data_dir = Path('/content/drive/MyDrive/SartoriousDatasets/supervised')

train_dir = data_dir/'train'
val_dir = data_dir/'val'

split_train_df = train_df[train_df.fold != 5]
split_val_df = train_df[train_df.fold == 5]

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    os.makedirs(val_dir)
    
# create train and validation images
# create_images(
#     split_train_df, train_dir/'images')
# create_images(
#     split_val_df, val_dir/'images')

# create train and validation binary masks
# create_masks(
#     split_train_df, train_dir/'masks', mask_type='binary')
# create_masks(
#     split_val_df, val_dir/'masks', mask_type='binary')

# # create train and validation 2-channel masks (mask + contour)
# create_masks(
#     split_train_df, train_dir/'masks_w_contours', mask_type='mask_w_contour')
# create_masks(
#     split_val_df, val_dir/'masks_w_contours', mask_type='mask_w_contour')

# # create train and validation categorical masks(bg = 0; binary mask = 1; contour = 2)
# create_masks(
#     split_train_df, train_dir/'categorical', mask_type='categorical')
# create_masks(
#     split_val_df, val_dir/'categorical', mask_type='categorical')

# create train and validation multiclass masks (mask + intersections (overlay masks included))
create_masks(
    split_train_df, train_dir/'categorical_intersection', mask_type='categorical_intersection')
create_masks(
    split_val_df, val_dir/'categorical_intersection', mask_type='categorical_intersection')

# # create train and validation instance masks
# create_instances(
#     split_train_df, train_dir/'instances')
# create_instances(
#     split_val_df, val_dir/'instances')

485it [03:08,  2.58it/s]
121it [00:45,  2.67it/s]


In [ ]:
# create full masks data (for submission train). Images take from downloaded 'train' directory
submission_dir = Path('/content/drive/MyDrive/SartoriousDatasets/Initial')

mask_dir = submission_dir/'train_masks'

if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)

create_masks(
    train_df, mask_dir, mask_type='mask_w_contour')

606it [03:30,  2.88it/s]


### Extract intersection based on data science bowl 2018 top-1 solution

In [ ]:
from skimage.morphology import square, dilation
from skimage.segmentation import watershed
from skimage import measure

In [ ]:
def extract_dsb2018_mask(labels):
    tmp = dilation(labels > 0, square(9))  
    tmp2 = watershed(tmp, labels, mask=tmp, watershed_line=True) > 0
    tmp = tmp ^ tmp2
    tmp = dilation(tmp, square(7))

    props = measure.regionprops(labels)
    msk0 = (labels > 0)
    msk0 = msk0.astype('uint8')

    msk1 = np.zeros_like(labels, dtype='bool')

    max_area = np.max([p.area for p in props])

    for y0 in range(labels.shape[0]):
        for x0 in range(labels.shape[1]):
            if not tmp[y0, x0]:
                continue
            if labels[y0, x0] == 0:
                if max_area > 4000:
                    sz = 6
                else:
                    sz = 3
            else:
                sz = 3
                try:
                    if props[labels[y0, x0] - 1].area < 300:
                        sz = 1
                    elif props[labels[y0, x0] - 1].area < 2000:
                        sz = 2
                except:
                    print(f'Label {labels[y0, x0]} was not found by region proposal. Assuming its size is too small')
                    sz = 1
            uniq = np.unique(labels[max(0, y0-sz):min(labels.shape[0], y0+sz+1), max(0, x0-sz):min(labels.shape[1], x0+sz+1)])
            if len(uniq[uniq > 0]) > 1:
                msk1[y0, x0] = True
                msk0[y0, x0] = 0

    msk1 = msk1.astype('uint8')

    msk = np.stack([msk0, msk1])
    msk = np.rollaxis(msk, 0, 3)

    return msk


def extract_dsb2018_mask_v2(labels):
    tmp = dilation(labels > 0, square(9))  
    tmp2 = watershed(tmp, labels, mask=tmp)
    
    intersection = np.full_like(tmp2, fill_value=0., dtype=np.uint8)

    tmp2_labels = np.unique(tmp2)[1:]  # no bg

    for lbl in tmp2_labels:
        mask_l = np.where(tmp2 == lbl, 1, 0).astype(np.uint8)
        contours_l, _ = cv2.findContours(mask_l, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(intersection, contours_l, -1, (1, 0, 0))

    tmp = intersection.copy()
    tmp = dilation(tmp, square(7))

    props = measure.regionprops(labels)
    msk0 = (labels > 0)
    msk0 = msk0.astype('uint8')

    msk1 = np.zeros_like(labels, dtype='bool')

    max_area = np.max([p.area for p in props])

    for y0 in range(labels.shape[0]):
        for x0 in range(labels.shape[1]):
            if not tmp[y0, x0]:
                continue
            if labels[y0, x0] == 0:
                if max_area > 4000:
                    sz = 6
                else:
                    sz = 3
            else:
                sz = 3
                try:
                    if props[labels[y0, x0] - 1].area < 300:
                        sz = 1
                    elif props[labels[y0, x0] - 1].area < 2000:
                        sz = 2
                except:
                    print(f'Label {labels[y0, x0]} was not found by region proposal. Assuming its size is too small')
                    sz = 1
            uniq = np.unique(labels[max(0, y0-sz):min(labels.shape[0], y0+sz+1), max(0, x0-sz):min(labels.shape[1], x0+sz+1)])
            if len(uniq[uniq > 0]) > 1:
                msk1[y0, x0] = True
                msk0[y0, x0] = 0

    msk1 = msk1.astype('uint8')

    msk = np.stack([msk0, msk1])
    msk = np.rollaxis(msk, 0, 3)

    return msk

In [ ]:
data_dir = Path('/content/drive/MyDrive/SartoriousDatasets/supervised')

train_dir = data_dir/'train'
val_dir = data_dir/'val'

split_train_df = train_df[train_df.fold != 5]
split_val_df = train_df[train_df.fold == 5]

def save_dsb2018_masks(dataset, save_dir, v2=False):
  if not os.path.exists(save_dir):
      os.makedirs(save_dir)

  for _, (path, annot, width, height) in tqdm(dataset[['img_path', 'annotation', 'width', 'height']].iterrows()):
      stem = Path(path).stem
      
      instance_mask = get_mask(annot, width, height, binary_mask=False)

      if v2:
        dsb2018_mask = extract_dsb2018_mask_v2(instance_mask)
      else:
        dsb2018_mask = extract_dsb2018_mask(instance_mask)

      dsb2018_path = Path(save_dir)/(stem + '_' + 'dsb2018_mask' + '.tif')

      Image.fromarray(dsb2018_mask).save(dsb2018_path)


save_dsb2018_masks(split_train_df, train_dir/'dsb2018_masks_v2', v2=True)
save_dsb2018_masks(split_val_df, val_dir/'dsb2018_masks_v2', v2=True)